# Instalation of libraries (if necessary)

In [1]:
!pip install tf_agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 17.4 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 10.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 5.0 MB/s eta 0:00:00a 0:00:01m
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697630 sha256=e04fc1e8240705c5d9ba72edc9cb64dbfd7d03b2f35f40f8a32bd3189992ecdf
  Stored in directory: /root/.cache/pip/wheels/bf/19/ce/d2b762b6d61115bf0b4260ca59650ba2d55d49f34f61e095f6
Successfully built gym
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
ERROR

# Libraries

In [47]:
import tensorflow as tf
from game import GameAI
from tools import *
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

In [45]:
state_shape = (4,) 
#State Shape
#Cards Dealer
#Cards Player
#Sum cards Player
#Sum cards Dealer

num_actions = (2,)
#0 Hit 
#1 Stand
#2 Split
#3 Doblar

# Info

## Outputs jugador
### 0 = Seguir
### 1 = Quedarse
### 2 = Split
### 3 = Doblar

## Estado ganador
### 0 Player Wins
### 1 Draw
### 2 Dealer Wins

# Hyperparameters

In [ ]:
num_iterations = 2000
initial_collect_steps = 100
collect_steps_per_iteration =   1
replay_buffer_max_length = 100000
batch_size = 64
learning_rate = 1e-3
log_interval = 200
num_eval_episodes = 10
eval_interval = 1000

amount_apuesta = 30

# Enviroments

In [ ]:
class Enviroment(py_environment.PyEnvironment):
    def __init__(self):
        self.Blackjack = GameAI(200000)
        self.observation_spec = []
    def action_spec(self):

    def observation_spec(self):
        cardsDealer = translate_array(self.Blackjack.getCardsDealer())
        cardsAI = translate_array(self.Blackjack.getCardsAI())
        sumDealer = countCards(cardsDealer)
        sumAI = countCards(cardsAI)

        observation_spec = np.concatenate(cardsDealer, cardsAI, sumDealer, sumAI)
    
    def _reset(self):
        self.Blackjack.__reset()
        self.Blackjack.setApuesta(amount_apuesta)
        self.Blackjack.setReady()
        self.Blackjack.__game()

# Agent

In [ ]:
class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()

    def network(self):
        learning_rate = 0.001
        init = tf.keras.initializers.HeUniform()
        model = keras.Sequential()
        model.add(keras.layers.Dense(24, input_shape=state_shape, activation='relu', kernel_initializer=init))
        model.add(keras.layers.Dense(12, activation='relu', kernel_initializer=init))
        model.add(keras.layers.Dense(action_shape, activation='linear', kernel_initializer=init))
        model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])
        return model

# Loop

In [ ]:
python_environment = Enviroment()
tf_env = tf_py_environment.TFPyEnvironment(python_environment)

In [ ]:
time_step = tf_env.reset()
rewards = []
steps = []

for _ in range(num_iterations):
    reward_t = 0
    steps_t = 0
    tf_env.reset()
    while True:
        